In [57]:
import numpy as np
import pandas as pd

In [58]:
portfolio_input_file = "../Datasets/portfolio.csv"
profile_input_file = "../Datasets/profile.csv"
transcript_input_file = "../Datasets/transcript.csv"

In [59]:
portfolio_raw = pd.read_csv(portfolio_input_file, index_col=0)
profile_raw = pd.read_csv(profile_input_file, index_col=0)
transcript_raw = pd.read_csv(transcript_input_file, index_col=0)

#### Reading Raw Datasets

In [60]:
portfolio_df = portfolio_raw.copy()
portfolio_df.info()
portfolio_df.head()

<class 'pandas.core.frame.DataFrame'>
Index: 10 entries, 0 to 9
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   reward      10 non-null     int64 
 1   channels    10 non-null     object
 2   difficulty  10 non-null     int64 
 3   duration    10 non-null     int64 
 4   offer_type  10 non-null     object
 5   id          10 non-null     object
dtypes: int64(3), object(3)
memory usage: 560.0+ bytes


,reward,channels,difficulty,duration,offer_type,id
0,10,"['email', 'mobile', 'social']",10,7,bogo,ae264e3637204a6fb9bb56bc8210ddfd
1,10,"['web', 'email', 'mobile', 'social']",10,5,bogo,4d5c57ea9a6940dd891ad53e9dbe8da0
2,0,"['web', 'email', 'mobile']",0,4,informational,3f207df678b143eea3cee63160fa8bed
3,5,"['web', 'email', 'mobile']",5,7,bogo,9b98b8c7a33c4b65b9aebfe6a799e6d9
4,5,"['web', 'email']",20,10,discount,0b1e1539f2cc45b7b9fa7c272da2e1d7


In [61]:
profile_df = profile_raw.copy()
profile_df.info()
profile_df.head()

<class 'pandas.core.frame.DataFrame'>
Index: 17000 entries, 0 to 16999
Data columns (total 5 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   gender            14825 non-null  object 
 1   age               17000 non-null  int64  
 2   id                17000 non-null  object 
 3   became_member_on  17000 non-null  int64  
 4   income            14825 non-null  float64
dtypes: float64(1), int64(2), object(2)
memory usage: 796.9+ KB


,gender,age,id,became_member_on,income
0,NaN,118,68be06ca386d4c31939f3a4f0e3dd783,20170212,NaN
1,F,55,0610b486422d4921ae7d2bf64640c50b,20170715,112000.0
2,NaN,118,38fe809add3b4fcf9315a9694bb96ff5,20180712,NaN
3,F,75,78afa995795e4d85b5d9ceeca43f5fef,20170509,100000.0
4,NaN,118,a03223e636434f42ac4c3df47e8bac43,20170804,NaN


In [62]:
profile_df["became_member_on"] = pd.to_datetime(profile_df["became_member_on"], format="%Y%m%d")
profile_df.head()

,gender,age,id,became_member_on,income
0,NaN,118,68be06ca386d4c31939f3a4f0e3dd783,2017-02-12,NaN
1,F,55,0610b486422d4921ae7d2bf64640c50b,2017-07-15,112000.0
2,NaN,118,38fe809add3b4fcf9315a9694bb96ff5,2018-07-12,NaN
3,F,75,78afa995795e4d85b5d9ceeca43f5fef,2017-05-09,100000.0
4,NaN,118,a03223e636434f42ac4c3df47e8bac43,2017-08-04,NaN


In [63]:
transcript_df = transcript_raw.copy()
transcript_df.info()
transcript_df.head()

<class 'pandas.core.frame.DataFrame'>
Index: 306534 entries, 0 to 306533
Data columns (total 4 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   person  306534 non-null  object
 1   event   306534 non-null  object
 2   value   306534 non-null  object
 3   time    306534 non-null  int64 
dtypes: int64(1), object(3)
memory usage: 11.7+ MB


,person,event,value,time
0,78afa995795e4d85b5d9ceeca43f5fef,offer received,{'offer id': '9b98b8c7a33c4b65b9aebfe6a799e6d9'},0
1,a03223e636434f42ac4c3df47e8bac43,offer received,{'offer id': '0b1e1539f2cc45b7b9fa7c272da2e1d7'},0
2,e2127556f4f64592b11af22de27a7932,offer received,{'offer id': '2906b810c7d4411798c6938adc9daaa5'},0
3,8ec6ce2a7e7949b1bf142def7d0e0586,offer received,{'offer id': 'fafdcd668e3743c1bb461111dcafc2a4'},0
4,68617ca6246f4fbc85e91a2a49552598,offer received,{'offer id': '4d5c57ea9a6940dd891ad53e9dbe8da0'},0


In [64]:
transcript_df["event"].value_counts()

event
transaction        138953
offer received      76277
offer viewed        57725
offer completed     33579
Name: count, dtype: int64

#### Offer Events

In [65]:
offer_completed_df = transcript_df.copy()[transcript_df["event"] == "offer completed"].reset_index(drop=True)
# Extract offer id
offer_completed_df["offer_id"] = [eval(value_dict).get("offer_id") for value_dict in offer_completed_df["value"]]
# Drop event and value columns
offer_completed_df = offer_completed_df.drop(["event", "value"], axis=1)
# Rename time column
offer_completed_df = offer_completed_df.rename(columns={"time":"completed_time"})
offer_completed_df.info()
offer_completed_df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 33579 entries, 0 to 33578
Data columns (total 3 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   person          33579 non-null  object
 1   completed_time  33579 non-null  int64 
 2   offer_id        33579 non-null  object
dtypes: int64(1), object(2)
memory usage: 787.1+ KB


,person,completed_time,offer_id
0,9fa9ae8f57894cc9a3b8a9bbe0fc1b2f,0,2906b810c7d4411798c6938adc9daaa5
1,fe97aa22dd3e48c8b143116a8403dd52,0,fafdcd668e3743c1bb461111dcafc2a4
2,629fc02d56414d91bca360decdfa9288,0,9b98b8c7a33c4b65b9aebfe6a799e6d9
3,676506bad68e4161b9bbaffeb039626b,0,ae264e3637204a6fb9bb56bc8210ddfd
4,8f7dd3b2afe14c078eb4f6e6fe4ba97d,0,4d5c57ea9a6940dd891ad53e9dbe8da0


In [66]:
offer_received_df = transcript_df.copy()[transcript_df["event"] == "offer received"].reset_index(drop=True)
# Extract offer id
offer_received_df["offer_id"] = [eval(value_dict).get("offer id") for value_dict in offer_received_df["value"]]
# Drop event and value columns
offer_received_df = offer_received_df.drop(["event", "value"], axis=1)
# Rename time column
offer_received_df = offer_received_df.rename(columns={"time":"received_time"})

offer_received_df.info()
offer_received_df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 76277 entries, 0 to 76276
Data columns (total 3 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   person         76277 non-null  object
 1   received_time  76277 non-null  int64 
 2   offer_id       76277 non-null  object
dtypes: int64(1), object(2)
memory usage: 1.7+ MB


,person,received_time,offer_id
0,78afa995795e4d85b5d9ceeca43f5fef,0,9b98b8c7a33c4b65b9aebfe6a799e6d9
1,a03223e636434f42ac4c3df47e8bac43,0,0b1e1539f2cc45b7b9fa7c272da2e1d7
2,e2127556f4f64592b11af22de27a7932,0,2906b810c7d4411798c6938adc9daaa5
3,8ec6ce2a7e7949b1bf142def7d0e0586,0,fafdcd668e3743c1bb461111dcafc2a4
4,68617ca6246f4fbc85e91a2a49552598,0,4d5c57ea9a6940dd891ad53e9dbe8da0


In [67]:
offer_viewed_df = transcript_df.copy()[transcript_df["event"] == "offer viewed"].reset_index(drop=True)
# Extract offer id
offer_viewed_df["offer_id"] = [eval(value_dict).get("offer id") for value_dict in offer_viewed_df["value"]]
# Drop event and value columns
offer_viewed_df = offer_viewed_df.drop(["event", "value"], axis=1)
# Rename time column
offer_viewed_df = offer_viewed_df.rename(columns={"time":"viewed_time"})

offer_viewed_df.info()
offer_viewed_df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 57725 entries, 0 to 57724
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   person       57725 non-null  object
 1   viewed_time  57725 non-null  int64 
 2   offer_id     57725 non-null  object
dtypes: int64(1), object(2)
memory usage: 1.3+ MB


,person,viewed_time,offer_id
0,389bc3fa690240e798340f5a15918d5c,0,f19421c1d4aa40978ebb69ca19b0e20d
1,d1ede868e29245ea91818a903fec04c6,0,5a8bc65990b245e5a138643cd4eb9837
2,102e9454054946fda62242d2e176fdce,0,4d5c57ea9a6940dd891ad53e9dbe8da0
3,02c083884c7d45b39cc68e1314fec56c,0,ae264e3637204a6fb9bb56bc8210ddfd
4,be8a5d1981a2458d90b255ddc7e0d174,0,5a8bc65990b245e5a138643cd4eb9837


##### Match offer viewed and offer completed to offer received entry

In [147]:
# Merge offer duration information
offer_received_df_merged = offer_received_df.merge(portfolio_df[["id", "duration"]], left_on="offer_id", right_on="id")
offer_received_df_merged["offer_end_time"] = offer_received_df_merged["received_time"] + (offer_received_df_merged["duration"] * 24)
offer_received_df_merged = offer_received_df_merged.drop(columns=["duration"])

offer_received_df_merged.info()
offer_received_df_merged.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 76277 entries, 0 to 76276
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   person          76277 non-null  object
 1   received_time   76277 non-null  int64 
 2   offer_id        76277 non-null  object
 3   id              76277 non-null  object
 4   offer_end_time  76277 non-null  int64 
dtypes: int64(2), object(3)
memory usage: 2.9+ MB


,person,received_time,offer_id,id,offer_end_time
0,0011e0d4e6b944f998e987f904e8c1e5,0,3f207df678b143eea3cee63160fa8bed,3f207df678b143eea3cee63160fa8bed,96
1,00a794f62b9a48beb58f8f6c02c2f1a6,0,3f207df678b143eea3cee63160fa8bed,3f207df678b143eea3cee63160fa8bed,96
2,00b3400e4ff64ee68ce9ada1d0c222f0,0,3f207df678b143eea3cee63160fa8bed,3f207df678b143eea3cee63160fa8bed,96
3,00c91f31f5f74e769fa7a359b63e1a9f,0,3f207df678b143eea3cee63160fa8bed,3f207df678b143eea3cee63160fa8bed,96
4,00e8d701c583461e81cc10053681a12b,0,3f207df678b143eea3cee63160fa8bed,3f207df678b143eea3cee63160fa8bed,96


In [148]:
# Sort by ascending time
offer_received_df_merged = offer_received_df_merged.sort_values(by=["received_time", "person", "offer_id"])
offer_viewed_df = offer_viewed_df.sort_values(by=["viewed_time", "person", "offer_id"])
offer_completed_df = offer_completed_df.sort_values(by=["completed_time", "person", "offer_id"])

In [131]:
from collections import defaultdict

# Intialise dictionary to store offer_viewed times with list as default value
offer_viewed_dict = defaultdict(list)
# Use (person_id, offer_id) as the key
# Append the offer_viewed timings to the list tagged to the key
for index, row in offer_viewed_df.iterrows():
    offer_viewed_dict[(row['person'], row['offer_id'])].append(row['viewed_time'])

# Intialise dictionary to store offer_completed times with list as default value
offer_completed_dict = defaultdict(list)
# Use (person_id, offer_id) as the key
# Append the offer_completed timings to the list tagged to the key
for index, row in offer_completed_df.iterrows():
    offer_completed_dict[(row['person'], row['offer_id'])].append(row['completed_time'])

# Initiaise dictionary to store offer records for each person
person_offer_metrics_dict = {}

# Extract offer_received information to the dictionary
for index, row in offer_received_df_merged.iterrows():
    person = row['person']
    offer_id = row['offer_id']
    received_time = row['received_time']
    offer_end_time = row["offer_end_time"]
    # Implement unqiue offer key as offer_id can appear more than once for the same person
    unique_offer_key = (offer_id, received_time, index) 
    
    # Add person as key in the dictionary
    if person not in person_offer_metrics_dict:
        person_offer_metrics_dict[person] = {}

    # Tag unique_offer_key to the person and fill in the offer received record in a sub-dictionary
    if unique_offer_key not in person_offer_metrics_dict[person]:
        person_offer_metrics_dict[person][unique_offer_key] = {
            'received_time': received_time,
            'offer_end_time': offer_end_time,
            'viewed_time': None,
            'completed_time': None,
        }
        
    # Match the offer_viewed time. None by default.
    if (person, offer_id) in offer_viewed_dict:
        # Enumerate list to obtain position of value in the list
        for i, view_time in enumerate(offer_viewed_dict[(person, offer_id)]):
            # Check if the offer was viewed after offer received and viewed within offer validity period
            if received_time <= view_time <= offer_end_time:
                # Update viewed_time
                person_offer_metrics_dict[person][unique_offer_key]['viewed_time'] = view_time
                # Remove matched offer_viewed time
                offer_viewed_dict[(person, offer_id)].pop(i)
                break

    # Match the offer_completed time. None by default.   
    if (person, offer_id) in offer_completed_dict:
        for i, complete_time in enumerate(offer_completed_dict[(person, offer_id)]):
            # Check if the offer was completed after offer received and completed within offer validity period
            if received_time <= complete_time <= offer_end_time:
                # Update completed_time
                person_offer_metrics_dict[person][unique_offer_key]['completed_time'] = complete_time
                # Remove matched offer_completed time
                offer_completed_dict[(person, offer_id)].pop(i)
                break

In [139]:
# Extract data from person_offer_metrics_dict into dataframe
offers_data = []

for person, offers in person_offer_metrics_dict.items():
    for offer_id, metrics in offers.items():
        offers_data.append({
            'person': person,
            'offer_id': offer_id[0],
            'received_time': metrics["received_time"],
            'viewed_time': metrics["viewed_time"],
            'completed_time': metrics["completed_time"],
            'offer_end_time': metrics["offer_end_time"],
            'unique_offer_id': offer_id,
        })

offers_df = pd.DataFrame(offers_data)

# Standardise numeric types to float
offers_df["received_time"] = offers_df["received_time"].astype(float)
offers_df["offer_end_time"] = offers_df["offer_end_time"].astype(float)

offers_df.info()
offers_df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 76277 entries, 0 to 76276
Data columns (total 7 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   person           76277 non-null  object 
 1   offer_id         76277 non-null  object 
 2   received_time    76277 non-null  float64
 3   viewed_time      56567 non-null  float64
 4   completed_time   33579 non-null  float64
 5   offer_end_time   76277 non-null  float64
 6   unique_offer_id  76277 non-null  object 
dtypes: float64(4), object(3)
memory usage: 4.1+ MB


,person,offer_id,received_time,viewed_time,completed_time,offer_end_time,unique_offer_id
0,0011e0d4e6b944f998e987f904e8c1e5,3f207df678b143eea3cee63160fa8bed,0.0,6.0,NaN,96.0,"(3f207df678b143eea3cee63160fa8bed, 0, 0)"
1,0011e0d4e6b944f998e987f904e8c1e5,5a8bc65990b245e5a138643cd4eb9837,336.0,354.0,NaN,408.0,"(5a8bc65990b245e5a138643cd4eb9837, 336, 17750)"
2,0011e0d4e6b944f998e987f904e8c1e5,9b98b8c7a33c4b65b9aebfe6a799e6d9,504.0,516.0,576.0,672.0,"(9b98b8c7a33c4b65b9aebfe6a799e6d9, 504, 35580)"
3,0011e0d4e6b944f998e987f904e8c1e5,2298d6c36e964ae4a3e7e9706d1fb8c2,168.0,186.0,252.0,336.0,"(2298d6c36e964ae4a3e7e9706d1fb8c2, 168, 62184)"
4,0011e0d4e6b944f998e987f904e8c1e5,0b1e1539f2cc45b7b9fa7c272da2e1d7,408.0,432.0,576.0,648.0,"(0b1e1539f2cc45b7b9fa7c272da2e1d7, 408, 72473)"


In [149]:
# Check if distribution of offer received time tallies with the original
offers_df["received_time"].describe() == offer_received_df_merged["received_time"].describe()

count     True
mean      True
std      False
min       True
25%       True
50%       True
75%       True
max       True
Name: received_time, dtype: bool

In [150]:
offers_df["received_time"].describe()["std"]

196.48954838819222

In [151]:
offer_received_df_merged["received_time"].describe()["std"]

196.48954838821655

In [152]:
# Check if distribution of offer completed time tallies with the original
offers_df["completed_time"].describe() == offer_completed_df["completed_time"].describe()

count     True
mean      True
std      False
min       True
25%       True
50%       True
75%       True
max       True
Name: completed_time, dtype: bool

In [153]:
offers_df["completed_time"].describe()["std"]

192.35261418719182

In [154]:
offer_completed_df["completed_time"].describe()["std"]

192.35261418719074

In [155]:
# Check if distribution of offer viewed time tallies with the original
offers_df["viewed_time"].describe() == offer_viewed_df["viewed_time"].describe()

count    False
mean     False
std      False
min       True
25%       True
50%       True
75%       True
max       True
Name: viewed_time, dtype: bool

In [156]:
offers_df["viewed_time"].describe()

count    56567.000000
mean       353.750314
std        199.139007
min          0.000000
25%        180.000000
50%        408.000000
75%        516.000000
max        714.000000
Name: viewed_time, dtype: float64

In [157]:
offer_viewed_df["viewed_time"].describe()

count    57725.000000
mean       354.290515
std        199.317684
min          0.000000
25%        180.000000
50%        408.000000
75%        516.000000
max        714.000000
Name: viewed_time, dtype: float64

Lower count of viewed_time as view times that were recorded after the offer duration were ignored. Not considered a valid view.